In [1]:
import os
import pandas as pd
import numpy as np
import librosa
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [2]:
audio_dataset_path = '/home/trupti/Desktop/Zense_Proj_ML/archive/Data/genres_original'
metadata = pd.read_csv(r'/home/trupti/Desktop/Zense_Proj_ML/archive/Data/features_30_sec.csv')

In [3]:
def feature_extractor(file):
    audio, sample_rate = librosa.load(file_name, res_type = 'kaiser_fast')
    mfccs_features = librosa.feature.mfcc(y = audio, sr = sample_rate, n_mfcc = 40)
    mfccs_scaled_features = np.mean(mfccs_features.T, axis = 0)
    return mfccs_scaled_features

In [4]:
extracted_features = []
for index_num, row in tqdm(metadata.iterrows()):
    try:
        final_class_labels = row["label"]
        file_name = os.path.join(os.path.abspath(audio_dataset_path), final_class_labels + '/', str(row["filename"]))
        data = feature_extractor(file_name)
        extracted_features.append([data, final_class_labels])
    except Exception as e:
        print(f"Error: {e}")
        continue

553it [00:23, 18.19it/s]/tmp/ipykernel_5630/1946415227.py:2: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sample_rate = librosa.load(file_name, res_type = 'kaiser_fast')
/home/trupti/Desktop/anaconda3/lib/python3.11/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
558it [00:23, 16.45it/s]

Error: 


1000it [00:42, 23.40it/s]


In [5]:
extracted_features_df = pd.DataFrame(extracted_features, columns = ['feature', 'class'])

In [6]:
extracted_features_df['class'].value_counts()

blues        100
classical    100
country      100
disco        100
hiphop       100
metal        100
pop          100
reggae       100
rock         100
jazz          99
Name: class, dtype: int64

In [7]:
X = np.array(extracted_features_df['feature'].tolist())
y = np.array(extracted_features_df['class'].tolist())

In [8]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.19, random_state=42)
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


# Print the shapes of the split datasets
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

Shape of X_train: (809, 40)
Shape of X_test: (190, 40)
Shape of y_train: (809,)
Shape of y_test: (190,)


In [9]:
rf_classifier = RandomForestClassifier(n_estimators=300, random_state=8)

In [10]:
rf_classifier.fit(X_train, y_train)

RandomForestClassifier(n_estimators=300, random_state=8)

In [11]:
rf_predictions = rf_classifier.predict(X_test)

In [12]:
rf_predictions

array(['pop', 'pop', 'disco', 'disco', 'country', 'jazz', 'reggae',
       'jazz', 'reggae', 'metal', 'disco', 'metal', 'hiphop', 'metal',
       'pop', 'blues', 'reggae', 'hiphop', 'rock', 'country', 'jazz',
       'rock', 'metal', 'disco', 'hiphop', 'blues', 'classical',
       'classical', 'reggae', 'disco', 'jazz', 'pop', 'metal', 'jazz',
       'metal', 'metal', 'classical', 'classical', 'metal', 'reggae',
       'pop', 'rock', 'disco', 'hiphop', 'classical', 'rock', 'country',
       'pop', 'metal', 'reggae', 'metal', 'jazz', 'jazz', 'blues',
       'blues', 'reggae', 'classical', 'jazz', 'jazz', 'pop', 'metal',
       'metal', 'reggae', 'classical', 'pop', 'disco', 'jazz',
       'classical', 'country', 'classical', 'country', 'classical',
       'metal', 'classical', 'disco', 'reggae', 'blues', 'pop', 'jazz',
       'rock', 'rock', 'reggae', 'rock', 'reggae', 'rock', 'hiphop',
       'metal', 'jazz', 'hiphop', 'hiphop', 'hiphop', 'hiphop', 'reggae',
       'classical', 'pop', '

In [13]:
rf_accuracy = accuracy_score(y_test, rf_predictions)
print("Random Forest Accuracy for testing dataset:", rf_accuracy)

Random Forest Accuracy for testing dataset: 0.6789473684210526


In [14]:
y_train_pred = rf_classifier.predict(X_train)
train_accuracy = accuracy_score(y_train, y_train_pred)
print("Random Forest Accuracy for training dataset:", train_accuracy)

Random Forest Accuracy for training dataset: 1.0
